## Prepare and read data

In [1]:
import gc
import itertools
from pathlib import Path
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

In [4]:
def feature_cache(reset=False):
    def _feature_cache(func):
        def wrapper(train_df, test_df, *args):
            func_name = func.__name__
            train_feat_path = Path("../feature") / f"train_{func_name}.pkl"
            test_feat_path = Path("../feature") / f"test_{func_name}.pkl"
            # if feature exists, load feature
            if train_feat_path.exists() and test_feat_path.exists() and not reset:
                train_feats = pd.read_pickle(train_feat_path).reset_index(drop=True)
                test_feats = pd.read_pickle(test_feat_path).reset_index(drop=True)
                train_df = pd.concat([train_df, train_feats], axis=1)
                test_df = pd.concat([test_df, test_feats], axis=1)
            # if not exists, make feature and save as pickle
            else:
                before_cols = train_df.columns.tolist()
                train_df, test_df = func(train_df, test_df, *args)
                after_cols = train_df.columns.tolist()
                new_cols = [c for c in after_cols if c not in before_cols]
                train_feats = train_df[new_cols]
                test_feats = test_df[new_cols]
                train_feats.to_pickle(train_feat_path)
                test_feats.to_pickle(test_feat_path)            
            return train_df, test_df
        return wrapper

    return _feature_cache

## Feature Engineering

In [5]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [6]:
original_cols = train_df.columns.tolist()

In [7]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [8]:
nume_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols))
    if c not in ["Salary", "No"] and "float" in train_df[c].dtype.name
]

In [9]:
cat_cols = [c for c in train_df.columns if c not in multi_cat_cols + nume_cols + ["Salary", "No"]]

### HandCraft feature

In [10]:
def hand_feature(df):
    df["ohe_main_cluster"] = 1
    df.loc[(df.DevType.fillna("").str.contains("Student"))
                | (df.SalaryType != "Yearly")
                | (df["Age"] == "Under 18 years old")
                | (df["CurrencySymbol"] == "INR"),
           "ohe_main_cluster"
    ] = 0
    return df

In [11]:
train_df = hand_feature(train_df)
test_df = hand_feature(test_df)

In [13]:
len(original_cols), len(cat_cols), len(nume_cols), len(multi_cat_cols)

(128, 65, 40, 21)

### Multi-category encoding

In [14]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols, dtype=np.int8)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols, dtype=np.int8)
    all_feat_df = pd.concat([train_feat_df, test_feat_df], axis=0, ignore_index=True)
    train_feat_df[f"sum_answer_{c}"] = (train_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    test_feat_df[f"sum_answer_{c}"] = (test_df[c].str.count(";") + 1).fillna(-1).astype(np.int8)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)
    # SVD
    svd = TruncatedSVD(n_components=2, random_state=2020)
    all_svd_feats = pd.DataFrame(svd.fit_transform(all_feat_df), columns=[f"svd_{c}_{ix}" for ix in range(2)])
    train_df = pd.concat([train_df, all_svd_feats.iloc[:len(train_df)]], axis=1)
    test_df = pd.concat([test_df, all_svd_feats.iloc[len(train_df):].reset_index(drop=True)], axis=1)

### Simple-category encoding

In [15]:
for c in cat_cols:
    train_df[c], uniques = pd.factorize(train_df[c], sort=True)
    test_df[c] = uniques.get_indexer(test_df[c])

### Reduce data memory

In [16]:
for c in nume_cols:
    if train_df[c].nunique() > 1000:
        train_df[c] = train_df[c].astype(np.float32)
        test_df[c] = test_df[c].astype(np.float32)
    else:
        train_df[c] = train_df[c].astype(np.float16)
        test_df[c] = test_df[c].astype(np.float16)

In [17]:
for c in cat_cols:
    if train_df[c].max() > 32767:
        train_df[c] = train_df[c].astype(np.int32)
        test_df[c] = test_df[c].astype(np.int32)
    else:
        train_df[c] = train_df[c].astype(np.int16)
        test_df[c] = test_df[c].astype(np.int16)            

In [18]:
train_df.shape, test_df.shape

((33857, 508), (11259, 507))

### Change Category Name

In [19]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

### Aggregation

In [20]:
def get_importance_cols(use_num=50):
    # basicなモデルのimportanceを読み込み
    importance_df = pd.read_csv("../data/importance/003_importance.csv")
    imp_feats = importance_df["feature"].iloc[:use_num].tolist()
    return imp_feats

In [21]:
imp_feats = get_importance_cols(use_num=50)
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c in nume_cols or c[:4] == "ohe_"]

In [22]:
len(imp_cat_cols), len(imp_nume_cols)

(32, 15)

In [23]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [24]:
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        # one-hotは同じカテゴリの場合がある
        if cat_col == nume_col:
            continue
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("mean").astype(np.float32)
        all_df[f"agg_std_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("std").astype(np.float32)
        all_df[f"agg_max_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("max").astype(np.float32)
        all_df[f"agg_min_{cat_col}_{nume_col}"] = \
            all_df.groupby(cat_col)[nume_col].transform("min").astype(np.float32)
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = \
            all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

58

In [25]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [26]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [27]:
# @feature_cache(reset=False)
def target_encoding(train_df, test_df):
    te_cols = [c for c in train_df.columns if c in cat_cols]
    for c in tqdm(te_cols):
        new_col = "te_" + c
        train_df[new_col] = 0
        test_df[new_col] = 0
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
    return train_df, test_df

In [28]:
train_df, test_df = target_encoding(train_df, test_df)

In [31]:
multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]

In [33]:
# @feature_cache(reset=False)
def multiple_target_encoding(train_df, test_df):
     # multiple target encoding
    multi_te_cols = [c for c in train_df.columns if c in cat_cols or c[:4] == "ohe_"]
    imp_feats = get_importance_cols(use_num=30)
    multi_te_cols = [c for c in multi_te_cols if c in imp_feats or "main" in c]
    combi_multi_te_cols = list(itertools.combinations(multi_te_cols, 2))   

    for col_a, col_b in tqdm(combi_multi_te_cols):
        new_col = "te_" + col_a + "__" + col_b
        train_df[new_col] = 0
        test_df[new_col] = 0
        train_df["tmp"] = train_df[col_a].fillna("").astype(str) + train_df[col_b].fillna("").astype(str)
        test_df["tmp"] = test_df[col_a].fillna("").astype(str) + test_df[col_b].fillna("").astype(str)
        for trn_idx, val_idx in fold_idx_list:
            mean_val = train_df.loc[trn_idx].groupby("tmp")["Salary"].mean().astype(np.float32)
            train_df.loc[val_idx, new_col] = train_df.loc[val_idx, "tmp"].map(mean_val)
            test_df.loc[:, new_col] += test_df.loc[:, "tmp"].map(mean_val) / len(fold_idx_list)
        train_df[new_col] = train_df[new_col].astype(np.float32)
        test_df[new_col] = test_df[new_col].astype(np.float32)
        del train_df["tmp"], test_df["tmp"]
        gc.collect()
    return train_df, test_df

In [34]:
train_df, test_df = multiple_target_encoding(train_df, test_df)

In [35]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

3595


## Training

In [94]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 64,
            'min_data_in_leaf': 80, 
            'max_depth': 4,
            "bagging_freq": 5,
            "bagging_fraction": 0.8,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.1,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [87]:
def lgb_train(train_df, fold_idx_list, use_cols, data_type="all"):
    importances = pd.DataFrame()
    oof_preds = np.zeros(len(train_df))
    models = []

    for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
        print(f"Fold {fold_i+1}")
        train_data = train_df.loc[trn_idx]
        valid_data = train_df.loc[val_idx]
        if data_type == "main":
            train_data = train_data[train_data.ohe_main_cluster == 1]
            valid_data = valid_data[valid_data.ohe_main_cluster == 1]
        elif data_type == "not_main":
            train_data = train_data[train_data.ohe_main_cluster == 0]
            valid_data = valid_data[valid_data.ohe_main_cluster == 0]
        train_dataset = lgb.Dataset(
            train_data[use_cols],
            label = train_data["Salary"]
        )
        valid_dataset = lgb.Dataset(
            valid_data[use_cols],
            label = valid_data["Salary"]
        )
        model = lgb.train(
                    lgb_params,
                    train_dataset,
                    3000,
                    valid_sets = [train_dataset, valid_dataset],
                    verbose_eval=100,
                    early_stopping_rounds = 100,
                    #feval = eval_f1,
                    #callbacks = [log_callback],
        )
        imp_df = pd.DataFrame()
        imp_df['feature'] = use_cols
        imp_df['gain'] = model.feature_importance(importance_type="gain")
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
        models.append(model)
    return importances, oof_preds, models

In [88]:
importances1, oof_preds1, models1 = lgb_train(train_df, fold_idx_list, use_cols, data_type="main")

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 24456.8	valid_1's rmse: 25265.2
[200]	training's rmse: 22417.2	valid_1's rmse: 23919.1
[300]	training's rmse: 21437.6	valid_1's rmse: 23469
[400]	training's rmse: 20763.1	valid_1's rmse: 23245.8
[500]	training's rmse: 20242	valid_1's rmse: 23103.5
[600]	training's rmse: 19791.4	valid_1's rmse: 22988.8
[700]	training's rmse: 19415	valid_1's rmse: 22905.1
[800]	training's rmse: 19048	valid_1's rmse: 22833.2
[900]	training's rmse: 18718.1	valid_1's rmse: 22784.3
[1000]	training's rmse: 18394.4	valid_1's rmse: 22730.7
[1100]	training's rmse: 18096.8	valid_1's rmse: 22714.7
[1200]	training's rmse: 17788.8	valid_1's rmse: 22703.5
[1300]	training's rmse: 17522.4	valid_1's rmse: 22656.9
[1400]	training's rmse: 17260.2	valid_1's rmse: 22643.5
[1500]	training's rmse: 17018.8	valid_1's rmse: 22621.3
[1600]	training's rmse: 16784.2	valid_1's rmse: 22607.7
[1700]	training's rmse: 16554	valid_1's rmse: 22604.

In [98]:
# regression
importances1, oof_preds1, models1 = lgb_train(train_df, fold_idx_list, use_cols, data_type="main")

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 22425.6	valid_1's rmse: 24046.1
[200]	training's rmse: 20858	valid_1's rmse: 23345
[300]	training's rmse: 19868	valid_1's rmse: 23123.1
[400]	training's rmse: 19100.3	valid_1's rmse: 22949.5
[500]	training's rmse: 18416.9	valid_1's rmse: 22816.9
[600]	training's rmse: 17847.6	valid_1's rmse: 22807.5
[700]	training's rmse: 17306.8	valid_1's rmse: 22791
[800]	training's rmse: 16744.2	valid_1's rmse: 22782.3
[900]	training's rmse: 16276.1	valid_1's rmse: 22741.3
[1000]	training's rmse: 15829.6	valid_1's rmse: 22755.9
Early stopping, best iteration is:
[904]	training's rmse: 16259.6	valid_1's rmse: 22738.1
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 22442.4	valid_1's rmse: 24238.4
[200]	training's rmse: 20850.1	valid_1's rmse: 23393.9
[300]	training's rmse: 19886.7	valid_1's rmse: 23114.5
[400]	training's rmse: 19102.5	valid_1's rmse: 22972.5
[500]	tra

In [95]:
importances2, oof_preds2, models2 = lgb_train(train_df, fold_idx_list, use_cols, data_type="not_main")

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 18431.9	valid_1's rmse: 19889.2
[200]	training's rmse: 17287.5	valid_1's rmse: 19545.3
[300]	training's rmse: 16606.8	valid_1's rmse: 19429.6
[400]	training's rmse: 16118.3	valid_1's rmse: 19370.2
[500]	training's rmse: 15655.3	valid_1's rmse: 19332.4
Early stopping, best iteration is:
[490]	training's rmse: 15693.9	valid_1's rmse: 19329.9
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 18464.9	valid_1's rmse: 19850.5
[200]	training's rmse: 17338.9	valid_1's rmse: 19383.7
[300]	training's rmse: 16645.8	valid_1's rmse: 19206
[400]	training's rmse: 16121.6	valid_1's rmse: 19104
[500]	training's rmse: 15653	valid_1's rmse: 19034.5
[600]	training's rmse: 15256.4	valid_1's rmse: 18985.5
[700]	training's rmse: 14892.6	valid_1's rmse: 18942.2
[800]	training's rmse: 14531.2	valid_1's rmse: 18914.6
[900]	training's rmse: 14190.9	valid_1's rmse: 18886
[1000]	tra

In [90]:
oof_preds1[:10]

array([24175.96168892, 84017.84797697, 64794.88652184, 26153.1749045 ,
       26015.47657692, 17822.07465059, 93981.38896443, 43522.56110271,
       50731.89406961, 46871.9808509 ])

In [91]:
oof_preds2[:10]

array([15602.82588989, 82554.57365917, 67828.71633595, 30019.97793802,
        9823.24383014, 15080.12691099, 95456.92299799, 46658.20816218,
       54379.22601847, 43352.13297764])

In [99]:
oof_preds = np.zeros(len(train_df))
main_idx = train_df["ohe_main_cluster"] == 0
oof_preds[main_idx] = oof_preds1[main_idx]
not_main_idx = train_df["ohe_main_cluster"] == 1
oof_preds[not_main_idx] = oof_preds2[not_main_idx]

In [100]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

22179.790443454753

In [80]:
importances1.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:300]

,feature,gain
0,te_Country__Age,1.758685e+13
1,te_Country__YearsCodingProf,8.619316e+12
2,te_Country__CareerSatisfaction,7.693014e+12
3,te_Country__YearsCoding,6.874477e+12
4,te_Country__ohe_Methodology_Agile,6.391237e+12
5,te_Country__SalaryType,5.522587e+12
6,te_Country__Employment,5.466526e+12
7,te_YearsCodingProf__CurrencySymbol,5.150380e+12
8,te_SalaryType__CurrencySymbol,4.387918e+12
9,te_Country__FormalEducation,4.380449e+12


## Prediction

In [40]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [41]:
test_pred

array([ 67201.099548  ,  89958.42362507,  85313.89621925, ...,
        88527.25668217,  81517.82690536, 115974.29261803])

In [42]:
sub_df = pd.read_csv("../input/submit.csv")

In [43]:
sub_df["Salary"] = test_pred

In [44]:
sub_df.to_csv("../predict/017_ohe_main_20296.csv", index=False)